# Sep 21, 2025: canonical systems
Joanes ICs and RSNs, 
but tesselated by the parcellation

conda env: gt

You want to "fit" or adjust the IC labels to the ROIs, ensuring that the ICs are re-mapped or deformed to match the parcellation regions. This could involve modifying the labels of the ICs based on the overlap between the IC map and the ROI parcellation.

In [1]:
import os
import nibabel as nib
import numpy as np
import pandas as pd

In [2]:
class ARGS():
    pass

args = ARGS()

args.SEED = 100

def set_seed(args):
    # gt.seed_rng(args.SEED)
    np.random.seed(args.SEED)

set_seed(args)

In [3]:
args.source = 'allen' #'spatial' #'allen'
args.space = 'ccfv2'
args.brain_div = 'whl'
args.num_rois = 172 #162 #172
args.resolution = 200

PARC_DESC = (
    f'source-{args.source}'
    f'_space-{args.space}'
    f'_braindiv-{args.brain_div}'
    f'_nrois-{args.num_rois}'
    f'_res-{args.resolution}'
)
PARC_DESC

'source-allen_space-ccfv2_braindiv-whl_nrois-172_res-200'

In [4]:
args.GRAPH_DEF = f'constructed'
args.GRAPH_METHOD = f'pearson'
args.THRESHOLD = f'signed'
args.EDGE_DEF = f'binary'
args.EDGE_DENSITY = 20
args.LAYER_DEF = f'individual'
args.DATA_UNIT = f'grp'

BASE_path = f'{os.environ["HOME"]}/new_mouse_dataset'
PARCELS_path = f'{BASE_path}/parcels'
IN_ICS_path = f'{BASE_path}/joanes_rsns'
SYS_path = f'{BASE_path}/canonical_systems/joanes/space-{args.space}'
os.system(f'mkdir -p {SYS_path}')
ROI_path = (
    f'{BASE_path}/roi-results-v3'
    f'/{PARC_DESC}'
)
ROI_SYS_path = f'{ROI_path}/canonical-systems/joanes'
os.makedirs(f'{ROI_SYS_path}', exist_ok=True)

In [5]:
parcels_img = nib.load(f'{PARCELS_path}/{PARC_DESC}_desc-parcels.nii.gz')
parcels_vol = parcels_img.get_fdata()
parcels_vol = np.round(parcels_vol, decimals=0)

rois_df = pd.read_csv(f'{PARCELS_path}/{PARC_DESC}_desc-names.csv')
roi_labels = rois_df['roi'].to_numpy()
roi_names = rois_df['name'].to_list()

In [6]:
vox_counts = np.zeros((len(rois_df),), dtype=int)
for roi in rois_df['roi'].to_list():
    roi_mask = parcels_vol == roi
    u, c = np.unique(roi_mask, return_counts=True)
    d = dict(zip(u, c))
    try: 
        vox_counts[roi-1] = d[True]
    except:
        pass
np.savetxt(f'{ROI_SYS_path}/desc-roi-vox-counts.csv', vox_counts, fmt="%d")
# vox_counts

In [7]:
lrois_df = rois_df[rois_df['name'].str.contains('L-', case=True, na=False)].reset_index(drop=True)
rrois_df = rois_df[rois_df['name'].str.contains('R-', case=True, na=False)].reset_index(drop=True)
rois_table_df = pd.DataFrame({
    'r-roi': rrois_df['roi'],
    'l-roi': lrois_df['roi'],
})
rois_table_df

,r-roi,l-roi
0,1,87
1,2,88
2,3,89
3,4,90
4,5,91
...,...,...
81,82,168
82,83,169
83,84,170
84,85,171


In [8]:
# remove cerebellar and hind brain regions because we don't have ICs for those regions
excluded_rois = rois_df[rois_df['name'].str.contains('Pons|Medulla|Cerebellar', case=False, na=False,)]['roi'].to_list()

excluded_vol = np.where(np.isin(parcels_vol, excluded_rois), parcels_vol, 0)
excluded_mask = np.where(excluded_vol, 1, 0)

rem_parcels_vol = np.where(np.isin(parcels_vol, excluded_rois), 0, parcels_vol)
rem_parcels_mask = np.where(parcels_vol, 1, 0)

In [9]:
excluded_rois

[79, 80, 81, 82, 83, 84, 85, 86, 165, 166, 167, 168, 169, 170, 171, 172]

ics

In [10]:
ICS_file = f'{IN_ICS_path}/RSNs_inAllenSpace.nii' 
#thanks Francesca for transforming Joanes's ICs into ccfv2 space!
ics_img = nib.load(ICS_file)
ics_vol = ics_img.get_fdata() #4D

In [11]:
thresh = 0
mask = np.abs(ics_vol) > thresh # threshold the ics
thresh_ics_vol = np.where(mask, ics_vol, -np.inf)

argmax_ic_vol = np.argmax(thresh_ics_vol, axis=3) + 1 # starting from 1, not 0
argmax_ic_vol[np.all(~mask, axis=3)] = 0 #3D

argmax_ic_vol *= rem_parcels_mask # remove ICs from those regions

In [12]:
def get_dominant_ic_per_roi(argmax_ic_vol, roi_mask, ):
    roi_ics = argmax_ic_vol[roi_mask]
    unique_ic, counts = np.unique(roi_ics, return_counts=True)
    dominant_ic = unique_ic[np.argmax(counts)]
    return dominant_ic

In [13]:
adjusted_argmax_vol = np.zeros_like(argmax_ic_vol)
for idx_row, row in rois_table_df.iterrows():
    lroi = row['l-roi']
    rroi = row['r-roi']
    lroi_mask = rem_parcels_vol == lroi
    rroi_mask = rem_parcels_vol == rroi
    
    if not lroi in excluded_rois:
        dominant_ic = get_dominant_ic_per_roi(argmax_ic_vol, lroi_mask)
    # else:
    #     dominant_ic = 0
    adjusted_argmax_vol[lroi_mask] = dominant_ic
    adjusted_argmax_vol[rroi_mask] = dominant_ic

In [14]:
adjusted_ics_img = nib.Nifti1Image(
    adjusted_argmax_vol.astype(np.int16),
    affine=ics_img.affine,
    header=ics_img.header,
)
nib.save(adjusted_ics_img, f'{ROI_SYS_path}/desc-ic-argmax-masks.nii.gz')

In [15]:
unique_ics = np.unique(adjusted_argmax_vol)
unique_ics = unique_ics[unique_ics > 0] # remove background

ics_vol = np.zeros_like(ics_vol)

In [16]:
IC_path = f'{ROI_SYS_path}/ics'
os.makedirs(IC_path, exist_ok=True)

for idx_ic, ic in enumerate(unique_ics):
    ics_vol[..., ic-1] = (adjusted_argmax_vol == ic).astype(np.int16) # binary mask

ics_img = nib.Nifti1Image(
    ics_vol, 
    affine=ics_img.affine,
    header=ics_img.header,
)
nib.save(
    ics_img, 
    f'{IC_path}/desc-ic-masks-combined.nii.gz'
)

In [17]:
df = []

df += [(
    0, 'Pir', 'olfactory'
)]
df += [(
    1, 'M1', 'sensory'
)]
df += [(
    2, f'S1 (vlp) + S2', 'somatosensory'
)]
df += [(
    3, 'S1 (hl + fl)', 'somatosensory'
)]
df += [(
    4, 'S1 (fl + bf)', 'somatosensory'
)]
df += [(
    5, 'S1 (bf)', 'somatosensory'
)]
df += [(
    6, 'V2 + RSD', 'sensory'
)]
df += [(
    7, 'Au (d + v)', 'sensory'
)]
df += [(
    8, 'Cg2', 'limbic'
)]
df += [(
    9, 'Cg1 + RS', 'limbic'
)]
df += [(
    10, 'HP (v)', 'limbic'
)]
df += [(
    11, 'HP (d)', 'limbic'
)]
df += [(
    12, 'CPu', 'basal ganglia'
)]
df += [(
    13, 'CPu + En', 'basal ganglia'
)]
df += [(
    14, 'Sep + Pal (v)', 'basal ganglia'
)]
df += [(
    15, 'Amyg', 'limbic'
)]
df += [(
    16, 'Th', 'limbic'
)]
df += [(
    17, 'MO + GI', 'olfactory'
)]
ics_df = pd.DataFrame(data=df, columns=['ic', 'name', 'rsn'])
ics_df.to_csv(f'{ROI_SYS_path}/desc-ic-names.csv')
ics_df

,ic,name,rsn
0,0,Pir,olfactory
1,1,M1,sensory
2,2,S1 (vlp) + S2,somatosensory
3,3,S1 (hl + fl),somatosensory
4,4,S1 (fl + bf),somatosensory
5,5,S1 (bf),somatosensory
6,6,V2 + RSD,sensory
7,7,Au (d + v),sensory
8,8,Cg2,limbic
9,9,Cg1 + RS,limbic


In [18]:
ics_vec = np.zeros((len(rois_df),), dtype=int)
for roi in rois_df['roi'].to_list():
    roi_mask = parcels_vol == roi
    try:
        dom_ic = np.unique(adjusted_argmax_vol[roi_mask])[0]
        ics_vec[roi-1] = int(dom_ic)
    except:
        pass

num_ics = np.max(ics_vec) + 1
roi_ics_mat = np.eye(num_ics)[ics_vec]
roi_ics_mat = roi_ics_mat[:, 1:] # removing the extra col containing unmapped rois

np.savetxt(f'{ROI_SYS_path}/desc-roi-ics-mat.csv', roi_ics_mat, fmt="%d") 
# ICs start from 0 index, 0 to 17, in the saved matrix

rsns

In [19]:
df = []

df += [(
    0, 'somatosensory', [2, 3, 4, 5],
)]
df += [(
    1, 'sensory', [1, 6, 7],
)]
df += [(
    2, 'olfactory', [0, 17],
)]
df += [(
    3, 'limbic', [9, 8, 10, 11, 15, 16],
)]
df += [(
    4, 'basal ganglia', [13, 12, 14],
)]
df += [(
    5, 'cerebellar', [],
)]
rsns_df = pd.DataFrame(data=df, columns=['rsn', 'name', 'ics'])
# rsns_df = rsns_df.sort_values(by='name').reset_index(drop=True)
rsns_df = rsns_df.reset_index(drop=True)
rsns_df.to_csv(f'{ROI_SYS_path}/desc-rsn-names.csv')
rsns_df

,rsn,name,ics
0,0,somatosensory,"[2, 3, 4, 5]"
1,1,sensory,"[1, 6, 7]"
2,2,olfactory,"[0, 17]"
3,3,limbic,"[9, 8, 10, 11, 15, 16]"
4,4,basal ganglia,"[13, 12, 14]"
5,5,cerebellar,[]


In [20]:
RSN_path = f'{ROI_SYS_path}/rsns'
os.makedirs(RSN_path, exist_ok=True)

rsns_vol = np.zeros((*adjusted_argmax_vol.shape, len(rsns_df)), dtype=np.int16)
for idx, row in rsns_df.iterrows():
    ics = row['ics']
    if len(ics) > 0:
        rsns_vol[..., idx] = np.sum(ics_vol[..., ics], axis=3)
    else:
        rsns_vol[..., idx] = excluded_mask

rsns_img = nib.Nifti1Image(
    rsns_vol, 
    affine=ics_img.affine,
    header=ics_img.header,
)
nib.save(rsns_img, f'{ROI_SYS_path}/desc-rsn-masks-combined.nii.gz')

In [21]:
for idx in range(rsns_vol.shape[-1]):
    rsn_vol = rsns_vol[..., idx]
    rsn_name = rsns_df.loc[idx, 'name']
    rsn_img = nib.Nifti1Image(
        rsn_vol,
        affine=rsns_img.affine,
        header=rsns_img.header,
    )
    nib.save(rsn_img, f'{RSN_path}/name-{rsn_name}_desc-{idx:02d}.nii.gz')

In [22]:
argmax_rsns_vol = np.argmax(rsns_vol, axis=3) + 1

In [23]:
rsns_vec = np.zeros((len(rois_df),), dtype=int)
for roi in rois_df['roi'].to_list():
    roi_mask = parcels_vol == roi
    try:
        dom_rsn = np.unique(argmax_rsns_vol[roi_mask])[0]
        rsns_vec[roi-1] = int(dom_rsn)
    except:
        pass

num_rsns = np.max(rsns_vec) + 1
roi_rsns_mat = np.eye(num_rsns)[rsns_vec]
roi_rsns_mat = roi_rsns_mat[:, 1:] # removing 0th col containing unmapped rois, but which cannot happen

np.savetxt(f'{ROI_SYS_path}/desc-roi-rsns-mat.csv', roi_rsns_mat, fmt="%d")
# RSNs start from 0 index: 0 to 5